In [1]:
import pandas as pd
import sqlite3

## Подключение к БД

In [3]:
conn = sqlite3.connect("data.db")
cursor = conn.cursor()

## Договоры

In [4]:
contracts = pd.read_sql_query("SELECT * FROM contract", conn)

contracts.head()

,guid,registration_number,notice_44fz,notice_not_placed_by_fz223p5s4,notice_44_num,lot_44_num,termination,extension,prolongation,customer_appeale_or_needs_approval,...,start_execution_date,end_execution_date,has_okpd_and_okdp_rows,has_okpd2_rows,is_electronic_place,electronic_place_name,electronic_place_url,electronic_place_publish_date,electronic_place_guid,region_name
0,000011fc-813d-466b-972a-a2a0fa251495,52320145827190000740175,0,0.0,31907889453 На право заключения договора поста...,None,0,0,0,0.0,...,2019-06-09,2020-12-30,0,1,NaN,None,None,None,None,Krasnodarskii_krai
1,00001ae1-4f7a-42c3-bec5-9520842e3e96,57727086772210001330001,0,1.0,None,None,0,0,0,0.0,...,2021-06-20,2021-08-15,0,1,NaN,None,None,None,None,Moskva
2,0000340e-b6cb-4c81-9b90-9f0ca6922f79,87725276584210000090001,0,1.0,None,None,0,0,0,0.0,...,2021-01-26,2021-02-09,0,1,NaN,None,None,None,None,Moskva
3,00005251-4234-4b39-9de1-87e63d6887bf,57702060116190000520003,0,1.0,None,None,0,0,0,0.0,...,2019-03-21,2021-09-29,0,1,NaN,None,None,None,None,Sankt-Peterburg
4,00005575-a566-4d95-b2c4-f62459667092,52320145827190000220048,0,0.0,31907493645 Поставка чернослива сушенного без ...,None,0,0,0,0.0,...,None,2021-01-30,0,1,NaN,None,None,None,None,Krasnodarskii_krai


### Разбиение договоров по регионам

In [6]:
contracts_count_by_region = contracts.groupby('region_name')["registration_number"].count()

contracts_count_by_region

region_name
Krasnodarskii_krai    104868
Leningradskaya_obl     33482
Moskovskaya_obl       203147
Moskva                948412
Primorskii_krai        33543
Sankt-Peterburg       209046
Name: registration_number, dtype: int64

### Ковидные договоры

In [7]:
covid_contracts = contracts[contracts["emergency"] == 1]
covid_contracts_count_by_region = covid_contracts.groupby('region_name')["registration_number"].count()

covid_contracts_count_by_region

region_name
Krasnodarskii_krai     313
Leningradskaya_obl     133
Moskovskaya_obl        800
Moskva                2375
Primorskii_krai         66
Sankt-Peterburg        796
Name: registration_number, dtype: int64

средняя сумма контракта

In [21]:
pc_avg_sum_by_reqion = purchase_contracts.groupby('region_name')['sum'].mean()

pc_avg_sum_by_reqion

region_name
Krasnodarskii_krai    8.762306e+06
Leningradskaya_obl    1.514665e+07
Moskovskaya_obl       7.999644e+06
Moskva                6.202049e+08
Primorskii_krai       1.023351e+07
Sankt-Peterburg       6.220484e+07
Name: sum, dtype: float64

сумма всех контрактов

In [22]:
pc_sum_by_reqion = purchase_contracts.groupby('region_name')['sum'].sum()

pc_sum_by_reqion

region_name
Krasnodarskii_krai    3.294627e+09
Leningradskaya_obl    3.968421e+09
Moskovskaya_obl       2.119906e+09
Moskva                2.977604e+12
Primorskii_krai       1.688529e+09
Sankt-Peterburg       5.996547e+10
Name: sum, dtype: float64

### по ОКВЭДам

In [23]:
contract_items = pd.read_sql_query(
    "SELECT * FROM contract_item",
    conn,
    dtype={'okved_code': str, 'okved_name': str, 'okved2_code': str, 'okved2_name': str}
)

contract_items.head()

,purchase_contract_guid,ordinal_number,okdp_code,okdp_name,okpd2_code,okpd2_name,okved_code,okved_name,okved2_code,okved2_name,okei_code,okei_name,qty,delivery_place_guid,additional_info
0,001aacfe-3914-4fe5-b488-841e8f77be2c,1,None,None,None,None,None,None,None,None,876,Условная единица,445.0,None,ОКПД2: 28.22.20.000 | ОКЕИ: 796; 839
1,001ab205-aa32-47b2-9c70-514c0421ba93,1,None,None,None,None,None,None,None,None,None,None,NaN,None,None
2,001ab205-aa32-47b2-9c70-514c0421ba93,10,None,None,None,None,None,None,None,None,None,None,NaN,None,None
3,001ab205-aa32-47b2-9c70-514c0421ba93,11,None,None,None,None,None,None,None,None,None,None,NaN,None,None
4,001ab205-aa32-47b2-9c70-514c0421ba93,12,None,None,None,None,None,None,None,None,None,None,NaN,None,None


In [37]:
purchase_contracts.index = purchase_contracts['guid']
contract_items.index = contract_items['purchase_contract_guid']

okved = contract_items[['okved_code', 'okved_name', 'okved2_code', 'okved2_name']].drop_duplicates()

purchase_contracts_okved = purchase_contracts.join(okved)
purchase_contracts_okved.head()

,guid,registration_number,create_date_time,contract_create_date,lot_guid,currency_code,sum,purchase_info_guid,placer_inn,customer_inn,...,additional_info,publication_date_time,contract_status_code,version,modification_description,region_name,okved_code,okved_name,okved2_code,okved2_name
001aacfe-3914-4fe5-b488-841e8f77be2c,001aacfe-3914-4fe5-b488-841e8f77be2c,31908349774-01,2019-12-20 06:30:19,2019-12-18,e783e941-5f90-475c-b709-06fd165b130c,RUB,7.316589e+06,autogenerated-c6dbc5fd-adc0-4563-9579-fcc5d668...,7708503727,7708503727,...,None,2019-12-20 06:52:22,P,1,None,Moskva,None,None,None,None
001ab205-aa32-47b2-9c70-514c0421ba93,001ab205-aa32-47b2-9c70-514c0421ba93,31807044509-06,2020-05-27 10:43:26,2019-06-29,5bc9f7e1-6e9b-e477-e100-00000a104614,RUB,6.057900e+05,autogenerated-f6dfc62b-90d9-4862-8bb9-37a44795...,7706613770,7706613770,...,None,2020-05-27 10:48:43,P,2,Изменение реквизитов наименование банка (поста...,Moskva,NaN,NaN,NaN,NaN
001d39b6-9307-49f1-bdaa-c583bd230c34,001d39b6-9307-49f1-bdaa-c583bd230c34,31604457799-13,2019-12-19 11:45:13,2017-04-09,da07e6fb-d41b-45f7-98d4-853d78b1e8c5,RUB,6.334580e+08,autogenerated-63e351b2-46d4-4865-9104-2b93a5a4...,7706413348,7706413348,...,None,2019-12-19 11:49:09,P,2,в соответствии с ЕОСЗ,Moskva,63.30.1,Организация комплексного туристического обслуж...,None,None
0022e8a3-ebd5-4567-969c-0548f6981f0b,0022e8a3-ebd5-4567-969c-0548f6981f0b,31908460452-01,2019-12-17 09:28:01,2019-12-16,5a6c4422-84f7-495d-96af-14b8cc50d762,RUB,9.901120e+06,autogenerated-f8227844-4940-4910-b9a3-37e14b02...,7734135124,7734135124,...,None,2019-12-17 11:32:25,P,1,None,Moskva,NaN,NaN,NaN,NaN
0023d126-a445-4194-99c3-9b7f3145449f,0023d126-a445-4194-99c3-9b7f3145449f,31908536773-01,2019-12-19 05:49:20,2019-12-15,b08eaf34-803c-4175-ba69-eb5f0b4f0437,RUB,1.016668e+05,autogenerated-9d985f3e-69a5-4e2f-b5ab-8e4e13e7...,7719555477,7719555477,...,None,2019-12-19 05:50:36,P,1,None,Moskva,NaN,NaN,NaN,NaN


количество контрактов на закупку

In [41]:
pc_count_by_okved = purchase_contracts_okved.groupby("okved_code", dropna=False)['registration_number'].count()

pc_count_by_okved

okved_code
01.12.1        1
01.13.21       1
02.01          1
02.01.1        1
05.02.2        1
            ... 
MA             1
NA             1
None           1
OA             1
NaN         6761
Name: registration_number, Length: 280, dtype: int64

средняя сумма контракта

In [42]:
pc_count_by_okved = purchase_contracts_okved.groupby("okved_code", dropna=False)["sum"].mean()

pc_count_by_okved

okved_code
01.12.1     1.343285e+05
01.13.21    1.343285e+05
02.01       1.561120e+07
02.01.1     1.031400e+07
05.02.2     1.742400e+07
                ...     
MA          2.040000e+05
NA          1.099362e+06
None        7.316589e+06
OA          3.980000e+05
NaN         4.173052e+08
Name: sum, Length: 280, dtype: float64

сумма всех контрактов

In [43]:
pc_count_by_okved = purchase_contracts_okved.groupby("okved_code", dropna=False)["sum"].sum()

pc_count_by_okved

okved_code
01.12.1     1.343285e+05
01.13.21    1.343285e+05
02.01       1.561120e+07
02.01.1     1.031400e+07
05.02.2     1.742400e+07
                ...     
MA          2.040000e+05
NA          1.099362e+06
None        7.316589e+06
OA          3.980000e+05
NaN         2.739191e+12
Name: sum, Length: 280, dtype: float64

### ковид - не ковид ??? как будем определять ???

количество контрактов на закупку

средняя сумма контракта

сумма всех контрактов

## Признаковое пространство для оценки рисков контракта

In [44]:
contracts = pd.read_sql_query("SELECT * FROM contract", conn)

contracts.head()

,guid,registration_number,notice_44fz,notice_not_placed_by_fz223p5s4,notice_44_num,lot_44_num,termination,extension,prolongation,customer_appeale_or_needs_approval,...,start_execution_date,end_execution_date,has_okpd_and_okdp_rows,has_okpd2_rows,is_electronic_place,electronic_place_name,electronic_place_url,electronic_place_publish_date,electronic_place_guid,region_name
0,000011fc-813d-466b-972a-a2a0fa251495,52320145827190000740175,0,0.0,31907889453 На право заключения договора поста...,None,0,0,0,0.0,...,2019-06-09,2020-12-30,0,1,NaN,None,None,None,None,Krasnodarskii_krai
1,00001ae1-4f7a-42c3-bec5-9520842e3e96,57727086772210001330001,0,1.0,None,None,0,0,0,0.0,...,2021-06-20,2021-08-15,0,1,NaN,None,None,None,None,Moskva
2,0000340e-b6cb-4c81-9b90-9f0ca6922f79,87725276584210000090001,0,1.0,None,None,0,0,0,0.0,...,2021-01-26,2021-02-09,0,1,NaN,None,None,None,None,Moskva
3,00005251-4234-4b39-9de1-87e63d6887bf,57702060116190000520003,0,1.0,None,None,0,0,0,0.0,...,2019-03-21,2021-09-29,0,1,NaN,None,None,None,None,Sankt-Peterburg
4,00005575-a566-4d95-b2c4-f62459667092,52320145827190000220048,0,0.0,31907493645 Поставка чернослива сушенного без ...,None,0,0,0,0.0,...,None,2021-01-30,0,1,NaN,None,None,None,None,Krasnodarskii_krai


In [45]:
contracts.columns

Index(['guid', 'registration_number', 'notice_44fz',
       'notice_not_placed_by_fz223p5s4', 'notice_44_num', 'lot_44_num',
       'termination', 'extension', 'prolongation',
       'customer_appeale_or_needs_approval',
       'customer_approval_or_antimonopoly_descision_date',
       'start_execution_term', 'end_execution_term', 'plan_position_guid',
       'url_eis', 'url_vsrz', 'url_kis_rmis', 'create_date_time',
       'customer_inn', 'placer_inn', 'detached_org_inn', 'publication_date',
       'contract_status_code', 'version', 'modification_description',
       'digital_purchase', 'digital_purchase_code', 'emergency', 'provider',
       'provider_code', 'change_contract', 'contract_reg_number', 'name',
       'contract_date', 'approve_date', 'purchase_notice_info_guid',
       'purchase_notice_info_number', 'lot_guid', 'subject_contract',
       'purchase_type_code', 'resume_date', 'has_subcontractor',
       'has_subcontractor_code', 'subcontractors_total', 'has_good_info',
   